In [1]:
import sys
import os
import json
from itertools import groupby
from operator import itemgetter
from typing import List, Tuple, Dict

import gin
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Input, Conv2D, Concatenate, MaxPool2D
from tensorflow.keras.applications import VGG16

sys.path.append('..')

from vehicle_nowcasting.data.data_loader import build_source_from_metadata, make_dataset, generate_anchors
from vehicle_nowcasting.data.generate_anchors import generate_anchors as gen_anch
from vehicle_nowcasting.utils import iou, plot_bb, change_box_order, compute_stride_from_receptive_field

%matplotlib inline

In [2]:
main_dir = '..'
data_dir = os.path.join(main_dir, 'data')
metadata = pd.read_csv('../data/metadata.csv')
train = metadata[metadata.split == 'train']
val = metadata[metadata.split == 'train']
label_map = json.load(open('../data/label_map.json', 'r'))

train_sources = build_source_from_metadata(train, label_map, data_dir)
val_sources = build_source_from_metadata(val, label_map, data_dir)

In [3]:
train_ds = make_dataset(train_sources, mode='rpn')
val_ds = make_dataset(val_sources, mode='rpn')

W0912 20:57:35.258732 140025648064320 deprecation.py:323] From /home/santiago/anaconda3/envs/CV/lib/python3.6/site-packages/tensorflow_core/python/ops/array_ops.py:1486: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


15
15


In [4]:
def RPN(inputs, k):
    x = Conv2D(256, kernel_size=(3, 3),  
               activation='relu',
               name='window')(inputs)
    cls = Conv2D(2 * k, kernel_size=(1, 1), name='cls_head')(x)
    reg = Conv2D(4 * k, kernel_size=(1, 1), name='reg_head')(x)
    return Concatenate()([cls, reg])

In [5]:
k = 4
inputs = Input(shape=(224, 224, 3))
backbone_model = VGG16(input_tensor=inputs,
                       weights='imagenet',
                       include_top=False)
#x = backbone_model.layers[-5].output
x = backbone_model.output
model = Model(inputs=inputs, outputs=RPN(x, k))

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [7]:
def l1_smooth(x):
    return tf.where(tf.math.abs(x) < 1, 0.5 * x * x, tf.math.abs(x) - 0.5)
        
def RPN_loss(k, y_true, y_pred):
    
    p_true = y_true[:, :, :, :(2 * k)]
    t_true = y_true[:, :, :, -(4 * k):]
    p_pred = y_pred[:, :, :, :(2 * k)]
    t_pred = y_pred[:, :, :, -(4 * k):]
    
    cls_loss = tf.keras.losses.binary_crossentropy(p_true, p_pred)
    reg_loss = tf.reduce_sum(l1_smooth(t_true - t_pred))
    return tf.math.reduce_sum(cls_loss) + tf.math.reduce_sum(p_true * reg_loss)

In [8]:
adam = tf.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=adam,
              loss=lambda y_true, y_pred: RPN_loss(20, y_true, y_pred)) # ?? 

In [9]:
hist = model.fit_generator(train_ds.batch(1), validation_data=val_ds.batch(1))

InvalidArgumentError: Incompatible shapes: [1,6,6,24] vs. [1,5,5,24] [Op:Mul] name: loss/concatenate_loss/mul/

In [4]:
img, y_true = next(iter(train_ds))

In [6]:
p_true = y_true[:, :, :(2 * 15)]
t_true = y_true[:, :, -(4 * 15):]

In [10]:
tf.reduce_sum(tf.cast(tf.cast(p_true, tf.int32), tf.float64) - p_true)

<tf.Tensor: id=1242, shape=(), dtype=float64, numpy=0.0>

In [31]:
tf.keras.losses.binary_crossentropy(p_true, p_pred)

<tf.Tensor: id=37950, shape=(1, 12, 12), dtype=float32, numpy=
array([[[0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.514165 ],
        [0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       ],
        [0.       , 0.514165 , 0.514165 , 0.       , 0.       ,
         0.       , 0.514165 , 0.       , 0.       , 0.       ,
         0.514165 , 0.       ],
        [0.       , 0.       , 0.       , 0.       , 0.       ,
         0.514165 , 0.514165 , 0.       , 0.       , 0.       ,
         0.       , 0.       ],
        [0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       ],
        [0.       , 0.       , 0.       , 0.       , 1.5424949,
         0.514165 , 0.       , 0.       , 0.       , 0.       ,
         

In [33]:
tf.cast(p_true, tf.float64) * tf.cast(tf.keras.losses.Huber()(t_true, t_pred), tf.float64)

<tf.Tensor: id=38016, shape=(1, 12, 12, 30), dtype=float64, numpy=
array([[[[     0.    ,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ],
         [     0.    ,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ],
         [     0.    ,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ],
         ...,
         [     0.    ,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ],
         [     0.    ,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ],
         [259603.4375,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ]],

        [[     0.    ,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ],
         [     0.    ,      0.    ,      0.    , ...,      0.    ,
               0.    ,      0.    ],
         [     0.    ,      0.    ,      0.    , ...,      0.    ,
               0. 